In [4]:
from modules.data_modules import MotionDataModule



cfg.DATA.batch_size = 128

dm = MotionDataModule(**cfg.DATA, tiny=False)
dm.setup()

Number of sequences: 16429, max length: 469, size: 1737.26 MB
Number of sequences: 1046, max length: 199, size: 110.61 MB
Number of sequences: 3064, max length: 343, size: 324.00 MB


In [5]:
cfg.DATA

{'seq_len': 420,
 'batch_size': 128,
 'file_list_paths': {'_train': '../../data/HumanML3D/HumanML3D/train_cleaned.txt',
  '_val': '../../data/HumanML3D/HumanML3D/val_cleaned.txt',
  '_test': '../../data/HumanML3D/HumanML3D/test_cleaned.txt'},
 '_motion_path': '../../data/HumanML3D/HumanML3D/new_joints'}

In [6]:
batch = next(iter(dm.train_dataloader()))
motion_seq, text_enc = batch

In [7]:
motion_seq.shape

torch.Size([128, 420, 22, 3])

In [8]:
# currently the dataloader takes 16 seconds to load a batch...
# this is too slow for training
# to speed it up; we need to make changes
# i now keep all the data in memory, and load it from there
# this now takes 6 seconds to load a batch
# this is still too slow
# i need to speed it up further


In [9]:
[(b.shape, b.dtype) for b in batch]

[(torch.Size([128, 420, 22, 3]), torch.float32),
 (torch.Size([128, 3, 250]), torch.int64)]

In [11]:
import torch
import torch.nn as nn
from torch import Tensor
from typing import List


def lengths_to_mask(lengths: List[int],
                    device: torch.device,
                    max_len: int = None) -> Tensor:
    lengths = torch.tensor(lengths, device=device)
    max_len = max_len if max_len else max(lengths)
    mask = torch.arange(max_len, device=device).expand(
        len(lengths), max_len) < lengths.unsqueeze(1)
    return mask


class TransformerEncoder(nn.Module):
    def __init__(self, d_model, nhead, num_layers, dim_feedforward=2048, dropout=0.1, activation='relu'):
        super(TransformerEncoder, self).__init__()
        self.latent_size = 1
        self.latent_dim = d_model
        
        # self.query_pos_encoder = nn.Embedding(1000, d_model)

        self.global_motion_token = nn.Parameter(
                torch.randn(self.latent_size * 2, self.latent_dim))


        self.skel_enc = nn.Linear(66, d_model)

        encoder_layer =  nn.TransformerEncoderLayer(
                d_model=d_model, 
                nhead=nhead, 
                dim_feedforward=dim_feedforward, 
                dropout=dropout, 
                activation=activation, 
                batch_first=False
                )
        encoder_norm = nn.LayerNorm(d_model)

        self.encoder = nn.TransformerEncoder(
            encoder_layer=encoder_layer,
            norm=encoder_norm,
            num_layers=num_layers)
        

        decoder_layer = nn.TransformerDecoderLayer(
            d_model=d_model, 
            nhead=nhead, 
            dim_feedforward=dim_feedforward, 
            dropout=dropout, 
            activation=activation, 
            batch_first=False
            )
        decoder_norm = nn.LayerNorm(d_model)

        self.decoder = nn.TransformerDecoder(
            decoder_layer=decoder_layer,
            norm=decoder_norm,
            num_layers=num_layers)
        
        self.final_layer = nn.Linear(d_model, 66)
        
    def forward(self, src: Tensor):
        dist, lengths = self.encode(src)
        z = self.reparameterize(dist[:1], dist[1:])
        output = self.decode(z, lengths)
        return output

    def encode(self, src):
        batch_size = src.shape[0]
        lengths = [len(feature) for feature in src]
        x = src.permute(1, 0, 2)  # (B, S, F) -> (S, B, F)
        device = x.device
        mask = lengths_to_mask(lengths, device)
        dist = torch.tile(self.global_motion_token[:, None, :], (1, batch_size, 1))

        dist_masks = torch.ones((batch_size, dist.shape[0]),
                                dtype=bool,
                                device=x.device)
        aug_mask = torch.cat((dist_masks, mask), 1)

        # adding the embedding token for all sequences
        x = self.skel_enc(x)
        xseq = torch.cat((dist, x), 0)
        # print(xseq.shape)
        # # xseq = self.query_pos_encoder(xseq)
        # print(xseq.shape)

        # xseq = self.query_pos_encoder(xseq)
        dist = self.encoder(xseq,
                            src_key_padding_mask=~aug_mask)[:dist.shape[0]]

        return dist, lengths
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z: Tensor, lengths: List[int]):
        # print('decoding')
        mask = lengths_to_mask(lengths, z.device)
        bs, nframes = mask.shape

        queries = torch.zeros(nframes, bs, self.latent_dim, device=z.device)


        # print(z.shape)
        # print(queries.shape)
        xseq = torch.cat((z, queries), axis=0)
        z_mask = torch.ones((bs, self.latent_size),
                            dtype=bool,
                            device=z.device)
        augmask = torch.cat((z_mask, mask), axis=1)
        # xseq = self.query_pos_decoder(xseq)
        output = self.decoder( xseq, xseq)[z.shape[0]:]
        # print(output.shape)
        output = self.final_layer(output)
        # zero for padded area
        output[~mask.T] = 0
        # Pytorch Transformer: [Sequence, Batch size, ...]
        feats = output.permute(1, 0, 2)
        return feats

print(motion_seq.shape)
motion_seq = motion_seq.reshape(motion_seq.shape[0], motion_seq.shape[1], -1)
print(motion_seq.shape)

latent_dim = 256
nhead = 8
num_layers = 6
d_model = 256
dim_feedforward = latent_dim

transformer = TransformerEncoder(d_model, nhead, num_layers, dim_feedforward)

out = transformer(motion_seq)
print(out.shape)

torch.Size([128, 420, 22, 3])
torch.Size([128, 420, 66])
torch.Size([128, 420, 66])


In [12]:
from torch.optim import AdamW
from time import time
from utils import plot_3d_motion_animation



optimizer = AdamW(transformer.parameters(), lr=1e-4)

loss_fn = nn.MSELoss()

def train_step(batch):
    motion_seq, text_enc = batch
    motion_seq = motion_seq.reshape(motion_seq.shape[0], motion_seq.shape[1], -1)
    optimizer.zero_grad()
    output = transformer(motion_seq)
    loss = loss_fn(output, motion_seq)
    loss.backward()
    optimizer.step()
    return loss.item()

n_epochs = 100

for epoch in range(n_epochs):
    for bn, batch in enumerate(dm.train_dataloader()):
        batch_start = time()
        loss = train_step(batch)
        print(f'Epoch {epoch},\tBatch {bn},\tLoss {loss}', f'\tTime {time()-batch_start}')
    print(f'Epoch {epoch} Loss {loss}')
    output = transformer(motion_seq)
    plot_3d_motion_animation(output[0].detach().cpu().numpy(), title='Generated Motion', save_path=f'output_{epoch}.mp4')
    
    

Epoch 0,	Batch 0,	Loss 1.2867529392242432 	Time 48.2876410484314
Epoch 0,	Batch 1,	Loss 1.1176049709320068 	Time 38.748793840408325
Epoch 0,	Batch 2,	Loss 0.8456929326057434 	Time 38.37087106704712
Epoch 0,	Batch 3,	Loss 0.9594648480415344 	Time 39.6554491519928
Epoch 0,	Batch 4,	Loss 0.6949453949928284 	Time 38.96312999725342
Epoch 0,	Batch 5,	Loss 0.7214362621307373 	Time 39.29506492614746
Epoch 0,	Batch 6,	Loss 0.7119171619415283 	Time 38.70934700965881
Epoch 0,	Batch 7,	Loss 0.7455111145973206 	Time 39.22153401374817
Epoch 0,	Batch 8,	Loss 0.564123272895813 	Time 38.63730025291443
Epoch 0,	Batch 9,	Loss 0.4911697208881378 	Time 40.80144262313843
Epoch 0,	Batch 10,	Loss 0.7143735289573669 	Time 42.58835577964783
Epoch 0,	Batch 11,	Loss 0.5333280563354492 	Time 40.67775797843933
Epoch 0,	Batch 12,	Loss 0.45885419845581055 	Time 40.33706283569336
Epoch 0,	Batch 13,	Loss 0.4927862584590912 	Time 38.58107805252075
Epoch 0,	Batch 14,	Loss 0.5787313580513 	Time 40.18187093734741
Epoch 0,	

KeyboardInterrupt: 

In [6]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from utils import plot_3d_motion_frames_multiple, plot_3d_motion_animation, plot_3d_motion_frames_multiple
from glob import glob
import matplotlib.pyplot as plt


activation_dict = {
    'tanh': nn.Tanh(),
    'leaky_relu': nn.LeakyReLU(),
    'relu': nn.ReLU(),
    'sigmoid': nn.Sigmoid(),
    'elu': nn.ELU(),
    'swish': nn.SiLU(),
    'mish': nn.Mish(),
    'softplus': nn.Softplus(),
    'softsign': nn.Softsign(),
    # 'bent_identity': nn.BentIdentity(),
    # 'gaussian': nn.Gaussian(),
    'softmax': nn.Softmax(),
    'softmin': nn.Softmin(),
    'softshrink': nn.Softshrink(),
    # 'sinc': nn.Sinc(),
}


class CustomLoss(nn.Module):
    def __init__(self, klDiv_weight=0.000002, relative_weight=1,  root_weight=1):
        super(CustomLoss, self).__init__()
        self.klDiv_weight = klDiv_weight

    
    def forward(self, rel_rec, rel, root_rec, root, mu, logvar):
        mse_rel = F.mse_loss(rel_rec, rel)
        mse_root = F.mse_loss(root_rec, root)
        klDiv = self.kl_divergence(mu, logvar)

        loss = {
            'mse_rel': mse_rel,
            'mse_root': mse_root,
            'klDiv': klDiv,
            'total': mse_rel + mse_root + self.klDiv_weight * klDiv
        }
        return loss
    
    
    def kl_divergence(self, mu, logvar):
        return -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    

# no activation class
class Identity(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x
    
class TransformerMotionAutoencoder_Chunked(nn.Module):
    """
    We want this class to recieve:
        pose0: (batch_size, 1, num_joints=22, 3)
        velocity_relative: (batch_size, seq_len-1, num_joints=22, 3)
        root_travel: (batch_size, seq_len, 1, 3)
    from the dataloader and output:
        the same pose0,  velocity_relative, root_travel
    """
    def __init__(
        self,
        cfg,

    ):
        super(TransformerMotionAutoencoder_Chunked, self).__init__()

        # data things
        self.seq_len = cfg.get("seq_len", 160)
        self.input_dim = cfg.get("input_dim", 66)
        
        # model things
        self.hidden_dim = cfg.get("hidden_dim", 1024)
        self.n_layers = cfg.get("n_layers", 8)
        self.n_heads = cfg.get("n_heads", 6)
        self.dropout = cfg.get("dropout", 0.10)
        self.latent_dim = cfg.get("latent_dim", 256)
        self.activation = cfg.get("activation", "relu")
        self.activation = activation_dict[self.activation]
        self.transformer_activation = cfg.get("transformer_activation", "gelu")
        self.output_layer = cfg.get("output_layer", "linear")
        self.clip = cfg.get("clip_grad_norm", 1)
        self.batch_norm = cfg.get("batch_norm", False)
        self.hindden_encoder_layer_widths = cfg.get("hidden_encoder_layer_widths", [256] * 3 )

        # training things
        self.lr = cfg.get("learning_rate", 1 * 1e-5)
        self.optimizer = cfg.get("optimizer", "AdamW")
        self.load = cfg.get("load", False)
        self.checkpoint_path = cfg.get("_checkpoint_path", None)

        # logging things
        self.save_animations = cfg.get("_save_animations", True)
        self.loss_function = CustomLoss(cfg.get("klDiv_weight", 0.000001))


        ##### MODEL #####
        # the strucure will be as follows:
        # 0. con
        ## 1. encoder
            ## chunked transformer
            ## concat
            ## linear layer
        ## reparameterize
        ## 2. decoder
            ## linear layer
            ## chunked transformer
            ## concat
        
        # vel transformer encoder
        cfg_vte = cfg.MODEL.vel_transformer_encoder
        self.vel_transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=66,
                nhead=cfg_vte.get("nhead", 6),
                dim_feedforward=cfg_vte.get("dim_feedforward", 1024),
                dropout=cfg_vte.get("dropout", 0.1),
                activation=cfg_vte.get("activation", "relu"),
            ),
            num_layers=cfg_vte.get("num_layers", 6),
        ) 

        # root transformer encoder
        cfg_rte = cfg.MODEL.root_travel_transformer_encoder
        self.root_travel_transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=3,
                nhead=cfg_rte.get("nhead", 3),
                dim_feedforward=cfg_rte.get("dim_feedforward", 512),
                dropout=cfg_rte.get("dropout", 0.1),
                activation=cfg_rte.get("activation", "relu"),
            ),
            num_layers=cfg_rte.get("num_layers", 6),
        )
        
        # linear encoder
        cfg_le = cfg.MODEL.linear_encoder
        self.linear_encoder = nn.Sequential()
        current_dim = (self.seq_len -1 ) * 66 + 3 * self.seq_len + 66
        in_dims = [current_dim] + cfg_le.get("hidden_encoder_layer_widths", [256] * 3)
        out_dims = cfg_le.get("hidden_encoder_layer_widths", [256] * 3) + [2 * self.latent_dim]
        for i, (in_dim, out_dim) in enumerate(zip(in_dims, out_dims)):
            self.linear_encoder.add_module(f"linear_{i}", nn.Linear(in_dim, out_dim))
            if i< len(in_dims) - 1:
                self.linear_encoder.add_module(f"activation_{i}", activation_dict[cfg_le.get("activation", "relu")])
                if self.batch_norm:
                    self.linear_encoder.add_module(f"batch_norm_{i}", nn.BatchNorm1d(out_dim))
                self.linear_encoder.add_module(f"dropout_{i}", nn.Dropout(cfg_le.get("dropout", 0.1)))


        # linear decoder
        cfg_ld = cfg.MODEL.linear_decoder
        self.linear_decoder = nn.Sequential()
        target_dim = current_dim
        current_dim = self.latent_dim
        in_dims = [current_dim] + cfg_ld.get("hidden_decoder_layer_widths", [256] * 3)
        out_dims = cfg_ld.get("hidden_decoder_layer_widths", [256] * 3) + [target_dim]
        for i, (in_dim, out_dim) in enumerate(zip(in_dims, out_dims)):
            self.linear_decoder.add_module(f"linear_{i}", nn.Linear(in_dim, out_dim))
            # if i>0:
            self.linear_decoder.add_module(f"activation_{i}", activation_dict[cfg_ld.get("activation", "relu")])
            if self.batch_norm:
                self.linear_decoder.add_module(f"batch_norm_{i}", nn.BatchNorm1d(out_dim))
            self.linear_decoder.add_module(f"dropout_{i}", nn.Dropout(cfg_ld.get("dropout", 0.1)))

        # vel transformer decoder
        cfg_vtd = cfg.MODEL.vel_transformer_decoder
        self.vel_transformer_decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(
                d_model=66,
                nhead=cfg_vtd.get("nhead", 6),
                dim_feedforward=cfg_vtd.get("dim_feedforward", 1024),
                dropout=cfg_vtd.get("dropout", 0.1),
                activation=cfg_vtd.get("activation", "relu"),
            ),
            num_layers=cfg_vtd.get("num_layers", 6),
        )
    
        # root transformer decoder
        cfg_rtd = cfg.MODEL.root_travel_transformer_decoder
        self.root_travel_transformer_decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(
                d_model=3,
                nhead=cfg_rtd.get("nhead", 3),
                dim_feedforward=cfg_rtd.get("dim_feedforward", 512),
                dropout=cfg_rtd.get("dropout", 0.1),
                activation=cfg_rtd.get("activation", "relu"),
            ),
            num_layers=cfg_rtd.get("num_layers", 6),
        )
    
    

    def encode(self, pose0,  velocity_relative, root_travel):
        # pose0: (batch_size, 1, num_joints=22, 3)
        # velocity_relative: (batch_size, seq_len-1, num_joints=22, 3)
        # root_travel: (batch_size, seq_len, 1, 3)

        # 1. encoder
        # chunked transformer for vel and roof
        # concat
        # linear layer
        pose0 = pose0.view(-1, 66)
        velocity_relative = velocity_relative.view(-1, self.seq_len-1, 66)
        root_travel = root_travel.view(-1, self.seq_len, 3)

        # vel
        vel = self.vel_transformer_encoder(velocity_relative,)
        # root
        root = self.root_travel_transformer_encoder(root_travel, )

        print('vel:', vel.shape)
        print('root:', root.shape)
        print('pose0:', pose0.shape)
        # concat
        x = torch.cat([nn.Flatten()(vel), nn.Flatten()(root), pose0], dim=1)
        x = self.linear_encoder(x)
        mu, logvar = torch.chunk(x, 2, dim=1)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu.__add__(eps.__mul__(std))
    
    def decode(self, z):
        # linear layer
        # chunked transformer
        # concat
        z = self.linear_decoder(z)
        # vel
        print(z.shape)
        pose0 = z[:, -66:].view(-1, 1, 22, 3)
        vel = z[:, :(self.seq_len-1) * 66].view(-1, self.seq_len-1, 22* 3)
        root = z[:, (self.seq_len-1) * 66:-66].view(-1, self.seq_len, 3)

        vel = self.vel_transformer_decoder(vel, vel)
        root = self.root_travel_transformer_decoder(root, root)

        return pose0, vel, root
    
    def forward(self, pose0,  velocity_relative, root_travel):
        mu, logvar = self.encode(pose0,  velocity_relative, root_travel)
        z = self.reparameterize(mu, logvar)
        pose0_rec, vel_rec, root_rec = self.decode(z)
        
        return pose0_rec, vel_rec, root_rec, mu, logvar
    
    
class TransformerMotionAutoencoder_Concatenated(nn.Module):
    """
    We want this class to recieve:
        pose0: (batch_size, 1, num_joints=22, 3)
        velocity_relative: (batch_size, seq_len-1, num_joints=22, 3)
        root_travel: (batch_size, seq_len, 1, 3)
    from the dataloader and output:
        the same pose0,  velocity_relative, root_travel
    """
    def __init__(
        self,
        cfg,

    ):
        super(TransformerMotionAutoencoder_Concatenated, self).__init__()

        # data things
        self.seq_len = cfg.get("seq_len", 160)
        self.input_dim = cfg.get("input_dim", 66)
        
        # model things
        self.hidden_dim = cfg.get("hidden_dim", 1024)
        self.n_layers = cfg.get("n_layers", 8)
        self.n_heads = cfg.get("n_heads", 6)
        self.dropout = cfg.get("dropout", 0.10)
        self.latent_dim = cfg.get("latent_dim", 256)
        self.activation = cfg.get("activation", "relu")
        self.activation = activation_dict[self.activation]
        self.transformer_activation = cfg.get("transformer_activation", "gelu")
        self.output_layer = cfg.get("output_layer", "linear")
        self.clip = cfg.get("clip_grad_norm", 1)
        self.batch_norm = cfg.get("batch_norm", False)
        self.hindden_encoder_layer_widths = cfg.get("hidden_encoder_layer_widths", [256] * 3 )

        # training things
        self.lr = cfg.get("learning_rate", 1 * 1e-5)
        self.optimizer = cfg.get("optimizer", "AdamW")
        self.load = cfg.get("load", False)
        self.checkpoint_path = cfg.get("_checkpoint_path", None)

        # logging things
        self.save_animations = cfg.get("_save_animations", True)
        self.loss_function = CustomLoss(cfg.get("klDiv_weight", 0.000001))


        ##### MODEL #####
        # the strucure will be as follows:
        # 0. concat
        ## 1. encoder
            ## linear
            ##  transformer
            ## flatten
            ## linear layer
        ## reparameterize
        ## 2. decoder
            ## linear layer
            ## chunked transformer
            ## concat

        cfg_lei = cfg.MODEL.CONCAT_TRANSFORMER.linear_encoder_input
        print(cfg_lei)
        self.linear_encoder_input = nn.Sequential()
        current_dim = (self.seq_len -1 ) * 66 + 3 * self.seq_len + 66
        in_dims = [current_dim] + cfg_lei.get("hidden_encoder_layer_widths", [256] * 3)
        out_dims = cfg_lei.get("hidden_encoder_layer_widths", [256] * 3) + [self.seq_len*cfg_lei.get("out_dim", 128)]
        for i, (in_dim, out_dim) in enumerate(zip(in_dims, out_dims)):
            self.linear_encoder_input.add_module(f"linear_{i}", nn.Linear(in_dim, out_dim))
            self.linear_encoder_input.add_module(f"activation_{i}", activation_dict[cfg_lei.get("activation", "relu")])
            if self.batch_norm:
                self.linear_encoder_input.add_module(f"batch_norm_{i}", nn.BatchNorm1d(out_dim))
            self.linear_encoder_input.add_module(f"dropout_{i}", nn.Dropout(cfg_lei.get("dropout", 0.1)))


        # transformer encoder
        cfg_te = cfg.MODEL.CONCAT_TRANSFORMER.transformer_encoder
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                d_model=cfg_lei.get("out_dim", 128),
                nhead=cfg_te.get("nhead", 8),
                dim_feedforward=cfg_te.get("dim_feedforward", 1024),
                dropout=cfg_te.get("dropout", 0.1),
                activation=cfg_te.get("activation", "relu"),
            ),
            num_layers=cfg_te.get("num_layers", 6),
        ) 
        # linear encoder
        cfg_le = cfg.MODEL.CONCAT_TRANSFORMER.linear_encoder_output
        self.linear_encoder = nn.Sequential()
        current_dim = self.seq_len * cfg_lei.get("out_dim", 128)
        in_dims = [current_dim] + cfg_le.get("hidden_encoder_layer_widths", [256] * 3)
        out_dims = cfg_le.get("hidden_encoder_layer_widths", [256] * 3) + [2 * self.latent_dim]
        for i, (in_dim, out_dim) in enumerate(zip(in_dims, out_dims)):
            self.linear_encoder.add_module(f"linear_{i}", nn.Linear(in_dim, out_dim))
            if i< len(in_dims) - 1:
                self.linear_encoder.add_module(f"activation_{i}", activation_dict[cfg_le.get("activation", "relu")])
                if self.batch_norm:
                    self.linear_encoder.add_module(f"batch_norm_{i}", nn.BatchNorm1d(out_dim))
                self.linear_encoder.add_module(f"dropout_{i}", nn.Dropout(cfg_le.get("dropout", 0.1)))


        # linear decoder
        cfg_ldi = cfg.MODEL.CONCAT_TRANSFORMER.linear_decoder_input
        self.linear_decoder_input = nn.Sequential()
        target_dim = current_dim
        current_dim = self.latent_dim
        in_dims = [current_dim] + cfg_ldi.get("hidden_decoder_layer_widths", [256] * 3)
        out_dims = cfg_ldi.get("hidden_decoder_layer_widths", [256] * 3) + [target_dim]
        for i, (in_dim, out_dim) in enumerate(zip(in_dims, out_dims)):
            self.linear_decoder_input.add_module(f"linear_{i}", nn.Linear(in_dim, out_dim))
            # if i>0:
            self.linear_decoder_input.add_module(f"activation_{i}", activation_dict[cfg_ldi.get("activation", "relu")])
            if self.batch_norm:
                self.linear_decoder_input.add_module(f"batch_norm_{i}", nn.BatchNorm1d(out_dim))
            self.linear_decoder_input.add_module(f"dropout_{i}", nn.Dropout(cfg_ldi.get("dropout", 0.1)))

        # transformer decoder
        cfg_td = cfg.MODEL.CONCAT_TRANSFORMER.transformer_decoder
        self.transformer_decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(
                d_model=cfg_lei.get("out_dim", 128),
                nhead=cfg_td.get("nhead", 8),
                dim_feedforward=cfg_td.get("dim_feedforward", 1024),
                dropout=cfg_td.get("dropout", 0.1),
                activation=cfg_td.get("activation", "relu"),
            ),
            num_layers=cfg_td.get("num_layers", 6),
        )

        # linear output
        cfg_ldo = cfg.MODEL.CONCAT_TRANSFORMER.linear_decoder_output
        self.linear_decoder_output = nn.Sequential()
        target_dim = (self.seq_len -1 ) * 66 + 3 * self.seq_len + 66
        current_dim = cfg_ldi.get("out_dim", 128) * self.seq_len
        in_dims = [current_dim] + cfg_ldo.get("hidden_decoder_layer_widths", [256] * 3)
        out_dims = cfg_ldo.get("hidden_decoder_layer_widths", [256] * 3) + [target_dim]

        for i, (in_dim, out_dim) in enumerate(zip(in_dims, out_dims)):
            self.linear_decoder_output.add_module(f"linear_{i}", nn.Linear(in_dim, out_dim))
            # if i>0:
            self.linear_decoder_output.add_module(f"activation_{i}", activation_dict[cfg_ldo.get("activation", "relu")])
            if self.batch_norm:
                self.linear_decoder_output.add_module(f"batch_norm_{i}", nn.BatchNorm1d(out_dim))
            self.linear_decoder_output.add_module(f"dropout_{i}", nn.Dropout(cfg_ldo.get("dropout", 0.1)))




    def encode(self, pose0,  velocity_relative, root_travel):
        # pose0: (batch_size, 1, num_joints=22, 3)
        # velocity_relative: (batch_size, seq_len-1, num_joints=22, 3)
        # root_travel: (batch_size, seq_len, 1, 3)

        # 1. encoder
        # concat
        # linear layer
        # transformer encoder

        x = torch.cat([nn.Flatten()(velocity_relative), nn.Flatten()(root_travel), nn.Flatten()(pose0)], dim=1)
        print(x.shape)
        x = self.linear_encoder_input(x)
        x = x.view(-1, self.seq_len, cfg.MODEL.CONCAT_TRANSFORMER.linear_encoder_input.get("out_dim", 128))
        print(x.shape)
        x = self.transformer_encoder(x)
        print(x.shape)
        x = x.view(-1, self.seq_len * cfg.MODEL.CONCAT_TRANSFORMER.linear_encoder_input.get("out_dim", 128))
        print(x.shape)
        x = self.linear_encoder(x)
        mu, logvar = torch.chunk(x, 2, dim=1)
        print(mu.shape, logvar.shape)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu.__add__(eps.__mul__(std))
    
    def decode(self, z):
        out_dim =  cfg.MODEL.CONCAT_TRANSFORMER.linear_encoder_input.get("out_dim", 128)
        print('z shape:     ', z.shape)

        z = self.linear_decoder_input(z)
        print('z shape:     ', z.shape)
        z = z.view(-1, self.seq_len, out_dim)
        print('z shape:     ', z.shape)

        z = self.transformer_decoder(z, z)
        print('z shape:     ', z.shape)
        z = z.view(-1, self.seq_len * out_dim)
        print('z shape:     ', z.shape)
        z = self.linear_decoder_output(z)
        # vel
        print(z.shape)
        pose0 = z[:, -66:].view(-1, 1, 22, 3)
        vel = z[:, :(self.seq_len-1) * 66].view(-1, self.seq_len-1, 22, 3)
        root = z[:, (self.seq_len-1) * 66:-66].view(-1, self.seq_len, 1,  3)


        return pose0, vel, root
    
    def forward(self, pose0,  velocity_relative, root_travel):
        mu, logvar = self.encode(pose0,  velocity_relative, root_travel)
        z = self.reparameterize(mu, logvar)
        pose0_rec, vel_rec, root_rec = self.decode(z)
        relative_motion = self.reconstruct(pose0_rec, vel_rec)
        return relative_motion, root_rec, mu, logvar

    def reconstruct(self, pose0,  velocity_relative):
        motion_less_root = torch.cumsum(torch.cat([pose0, velocity_relative], dim=1), dim=1)
        return motion_less_root
            

# try it out on batch
def reconstruct(pose0,  velocity_relative):
    motion_less_root = torch.cumsum(torch.cat([pose0, velocity_relative], dim=1), dim=1)
    return motion_less_root
from config import config as cfg # to force reload: del cfg; from config import config as cfg
model = TransformerMotionAutoencoder_Concatenated(cfg)  

pose0,  velocity_relative, root_travel, motion_less_root, motion_seq = batch
rel = reconstruct(pose0,  velocity_relative)
print('pose0:', pose0.shape)
print('velocity_relative:', velocity_relative.shape)
print('root_travel:', root_travel.shape)

rel_rec, root_rec, mu, logvar = model(pose0,  velocity_relative, root_travel)
# get loss
loss = model.loss_function(rel_rec, rel, root_rec, root_travel, mu, logvar)
print(loss)

{'hidden_layer_widths': [256, 256, 256], 'dropout': 0.1, 'activation': 'relu', 'out_dim': 128}
pose0: torch.Size([2, 1, 22, 3])
velocity_relative: torch.Size([2, 159, 22, 3])
root_travel: torch.Size([2, 160, 1, 3])
torch.Size([2, 11040])
torch.Size([2, 160, 128])
torch.Size([2, 160, 128])
torch.Size([2, 20480])
torch.Size([2, 256]) torch.Size([2, 256])
z shape:      torch.Size([2, 256])
z shape:      torch.Size([2, 20480])
z shape:      torch.Size([2, 160, 128])
z shape:      torch.Size([2, 160, 128])
z shape:      torch.Size([2, 20480])
torch.Size([2, 11040])
{'mse_rel': tensor(5.1785, grad_fn=<MseLossBackward0>), 'mse_root': tensor(0.5949, grad_fn=<MseLossBackward0>), 'klDiv': tensor(1.2031, grad_fn=<MulBackward0>), 'total': tensor(5.7735, grad_fn=<AddBackward0>)}


In [7]:
cfg.MODEL.CONCAT_TRANSFORMER.linear_decoder_input

{'hidden_layer_widths': [256, 256, 256], 'dropout': 0.1, 'activation': 'relu'}

In [1]:
import torch
checkpoint_path = '../tb_logs5/TransformerMotionAutoencoder/version_85/checkpoints/epoch=34-step=4515.ckpt'
weights = torch.load(checkpoint_path)

In [16]:
print(len(weights['state_dict'].keys()))
enc_weight_keys = list(weights['state_dict'].keys())[:70]
dec_weight_keys = list(weights['state_dict'].keys())[70:]
for key in dec_weight_keys:
    print(key, weights['state_dict'][key].shape)

170
decoder_linear_block.0.weight torch.Size([1024, 1024])
decoder_linear_block.0.bias torch.Size([1024])
decoder_linear_block.3.weight torch.Size([4096, 1024])
decoder_linear_block.3.bias torch.Size([4096])
decoder_linear_block.6.weight torch.Size([8192, 4096])
decoder_linear_block.6.bias torch.Size([8192])
decoder_linear_block.9.weight torch.Size([10560, 8192])
decoder_linear_block.9.bias torch.Size([10560])
transformer_decoder.layers.0.self_attn.in_proj_weight torch.Size([198, 66])
transformer_decoder.layers.0.self_attn.in_proj_bias torch.Size([198])
transformer_decoder.layers.0.self_attn.out_proj.weight torch.Size([66, 66])
transformer_decoder.layers.0.self_attn.out_proj.bias torch.Size([66])
transformer_decoder.layers.0.multihead_attn.in_proj_weight torch.Size([198, 66])
transformer_decoder.layers.0.multihead_attn.in_proj_bias torch.Size([198])
transformer_decoder.layers.0.multihead_attn.out_proj.weight torch.Size([66, 66])
transformer_decoder.layers.0.multihead_attn.out_proj.bias

In [5]:
from typing import List
import torch
from torch import Tensor

def lengths_to_mask(lengths: List[int],
                    device: torch.device,
                    max_len: int = None) -> Tensor:
    """
    Provides a mask, of length max_len or the longest element in lengths. With True for the elements less than the length for each length in lengths.
    """
    lengths = torch.tensor(lengths, device=device)
    max_len = max_len if max_len else max(lengths)
    mask = torch.arange(max_len, device=device).expand(len(lengths), max_len) < lengths.unsqueeze(1)
    return mask

lengths = [10, 20, 30]

mask = lengths_to_mask(lengths, torch.device('cpu'))
mask


tensor([[ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
         False, False, False, False, False, False, False, False, False, False],
        [ True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True,  True,  True,  True,  True,  True,  True,  True]])